In [35]:
import numpy as np
import pandas as pd

In [58]:
new_place_data = pd.read_csv("ConjointInput.csv", sep = ";")

Input = a ranking of 3 different features (TV Size, TV Type, TV Color) with 3(32",37",42")<br>
, 2 (LCD, Plasma), 3(Black, Silver, Anthrazit) different stimuli types.

In [59]:

new_place_data

,Stimulus,Rank
0,A1B1C1,2
1,A1B1C2,3
2,A1B1C3,1
3,A1B2C1,5
4,A1B2C2,6
5,A1B2C3,4
6,A2B1C1,8
7,A2B1C2,9
8,A2B1C3,7
9,A2B2C1,11


## First step is to introduce dummy variables for every stimulus<p>
There are in total 9 differen stimuli, and 18 different combinations


In [43]:
ConjointDummyDF = pd.DataFrame(np.zeros((18,9)), columns=["Rank","A1", "A2", "A3",
                                                    "B1","B2", 
                                                    "C1", "C2",
                                                    "C3"])

In [61]:
def matrix_mapping(data,variable):
    if str(variable) in data.Stimulus:
        return 1
    else: return 0

parse_columns=["A1", "A2", "A3","B1","B2", "C1", "C2","C3"]
for column in parse_columns:
    new_place_data[column] = new_place_data.apply(lambda s: matrix_mapping(s, column), axis=1)


#new_place_data.drop('Stimulus', inplace=True, axis=1)
new_place_data

,Stimulus,Rank,A1,A2,A3,B1,B2,C1,C2,C3
0,A1B1C1,2,1,0,0,1,0,1,0,0
1,A1B1C2,3,1,0,0,1,0,0,1,0
2,A1B1C3,1,1,0,0,1,0,0,0,1
3,A1B2C1,5,1,0,0,0,1,1,0,0
4,A1B2C2,6,1,0,0,0,1,0,1,0
5,A1B2C3,4,1,0,0,0,1,0,0,1
6,A2B1C1,8,0,1,0,1,0,1,0,0
7,A2B1C2,9,0,1,0,1,0,0,1,0
8,A2B1C3,7,0,1,0,1,0,0,0,1
9,A2B2C1,11,0,1,0,0,1,1,0,0


In [62]:
final_table = new_place_data.drop('Stimulus', axis=1)
final_table

,Rank,A1,A2,A3,B1,B2,C1,C2,C3
0,2,1,0,0,1,0,1,0,0
1,3,1,0,0,1,0,0,1,0
2,1,1,0,0,1,0,0,0,1
3,5,1,0,0,0,1,1,0,0
4,6,1,0,0,0,1,0,1,0
5,4,1,0,0,0,1,0,0,1
6,8,0,1,0,1,0,1,0,0
7,9,0,1,0,1,0,0,1,0
8,7,0,1,0,1,0,0,0,1
9,11,0,1,0,0,1,1,0,0


In [40]:
ConjointDummyDF.head()

,Rank,A1,A2,A3,B1,B2,C1,C2,C3
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Insert the proper Stimulus names

In [63]:
fullNames = {"Rank":"Rank", \
           "A1": "1.crowdfunding","A2": "1.dzialania dla lokalnej spolecznosci","A3": "1.konkurs", \
          "B1": "2.caloroczny", "B2":"2.sezonowy", \
           "C1":"3.pawilon", "C2":"3.plac zabaw", "C3": "3.biuro",\
          }

final_table.rename(columns=fullNames, inplace=True)

In [64]:
final_table.head()

,Rank,1.crowdfunding,1.dzialania dla lokalnej spolecznosci,1.konkurs,2.caloroczny,2.sezonowy,3.pawilon,3.plac zabaw,3.biuro
0,2,1,0,0,1,0,1,0,0
1,3,1,0,0,1,0,0,1,0
2,1,1,0,0,1,0,0,0,1
3,5,1,0,0,0,1,1,0,0
4,6,1,0,0,0,1,0,1,0


## Estimate Main Effects with a linear regression
<p>

There are different ways for parameter estimation beside linear regression depending on what kind of rating you have.<br>
For example using Probit or Logit is the output is not a rank but a decision (1=chose stimulus, 0 = no choice).

In [65]:
import statsmodels.api as sm

In [66]:
final_table.columns

Index(['Rank', '1.crowdfunding', '1.dzialania dla lokalnej spolecznosci',
       '1.konkurs', '2.caloroczny', '2.sezonowy', '3.pawilon', '3.plac zabaw',
       '3.biuro'],
      dtype='object')

In [67]:
X = final_table[['1.crowdfunding', '1.dzialania dla lokalnej spolecznosci',
       '1.konkurs', '2.caloroczny', '2.sezonowy', '3.pawilon', '3.plac zabaw',
       '3.biuro']]
X = sm.add_constant(X)
Y = final_table.Rank
linearRegression = sm.OLS(Y, X). fit()
linearRegression.summary()

C:\Users\admin001\anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Rank   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 6.035e+30
Date:                Mon, 27 Sep 2021   Prob (F-statistic):          5.80e-182
Time:                        16:59:49   Log-Likelihood:                 574.82
No. Observations:                  18   AIC:                            -1138.
Df Residuals:                      12   BIC:                            -1132.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                     4.3846   4.36e-16   1.01e+16      0.000       4.385       4.385
1.crowdfunding                           -4.5385   1.34e-15  -3.38e+15      0.000      -4.538      -4.538
1.dzialania dla lokalnej spolecznosci     1.4615   1.34e-15   1.09e+15      0.000       1.462       1.462
1.konkurs                                 7.4615   1.34e-15   5.55e+15      0.000       7.462       7.462
2.caloroczny                              0.6923   9.69e-16   7.14e+14      0.000       0.692       0.692
2.sezonowy                                3.6923   9.69e-16   3.81e+15      0.000       3.692       3.692
3.pawilon                                 1.4615   1.34e-15   1.09e+15      0.000       1.462       1.462
3.plac zabaw                              2.4615   1.34e-15   1.83e+15      0.000       2.462       2.462
3.biuro                                   0.4615   1.34e-15   3.44e+14      0.000       0.462       0.462
==============================================================================
Omnibus:                        0.066   Durbin-Watson:                   0.997
Prob(Omnibus):                  0.968   Jarque-Bera (JB):                0.128
Skew:                          -0.093   Prob(JB):                        0.938
Kurtosis:                       2.632   Cond. No.                     4.62e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.83e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Part worth values & relative importance of the stimuli
<p>
Importance of Stimuli= Max(beta) - Min(beta)
<br>
Relative Importance of Stimuli = Importance of Stim / Sum(Importance of all Stimuli)

In [68]:
importance = []
relative_importance = []

rangePerFeature = []

begin = "A"
tempRange = []
for stimuli in fullNames.keys():
    if stimuli[0] == begin:
        tempRange.append(linearRegression.params[fullNames[stimuli]])
    elif stimuli == "Rank":
        rangePerFeature.append(tempRange)
    else:
        rangePerFeature.append(tempRange)
        begin = stimuli[0]
        tempRange = [linearRegression.params[fullNames[stimuli]]]
        

In [69]:
for item in rangePerFeature:
    importance.append( max(item) - min(item))

for item in importance:
    relative_importance.append(100* round(item/sum(importance),3))

In [71]:

partworths = []

item_levels = [1,3,5,8]

for i in range(1,4):
    part_worth_range = linearRegression.params[item_levels[i-1]:item_levels[i]]
    print(part_worth_range)

1.crowdfunding                          -4.538462
1.dzialania dla lokalnej spolecznosci    1.461538
dtype: float64
1.konkurs       7.461538
2.caloroczny    0.692308
dtype: float64
2.sezonowy      3.692308
3.pawilon       1.461538
3.plac zabaw    2.461538
dtype: float64


In [73]:
meanRank = []
for i in final_table.columns[1:]:
    newmeanRank = final_table["Rank"].loc[final_table[i] == 1].mean()
    meanRank.append(newmeanRank)

    
#total Mean or, "basic utility" is used as the "zero alternative"
totalMeanRank = sum(meanRank) / len(meanRank)



partWorths = {}
for i in range(len(meanRank)):
    name = fullNames[sorted(fullNames.keys())[i]]
    partWorths[name] = meanRank[i] - totalMeanRank

In [74]:
partWorths

{'1.crowdfunding': -6.0,
 '1.dzialania dla lokalnej spolecznosci': 0.0,
 '1.konkurs': 6.0,
 '2.caloroczny': -1.5,
 '2.sezonowy': 1.5,
 '3.pawilon': 0.0,
 '3.plac zabaw': 1.0,
 '3.biuro': -1.0}

### Summary & Results

In [82]:
print ("Relative Importance of Feature:\n\n1.dzialania towarzyszace",relative_importance[0], "%",\
"\n2.sezonowosc:", relative_importance[1], "%", "\n3.funkcja:", relative_importance[2], "%\n\n")

print ("--"*30)

print ("Importance of Feature:\n\n1.dzialania towarzyszace",importance[0],\
"\n2.sezonowosc:", importance[1],  "\n3.funkcja::", importance[2])

Relative Importance of Feature:

1.dzialania towarzyszace 44.4 % 
2.sezonowosc: 44.4 % 
3.funkcja: 11.1 %


------------------------------------------------------------
Importance of Feature:

1.dzialania towarzyszace 11.999999999999996 
2.sezonowosc: 11.999999999999996 
3.funkcja:: 3.0000000000000027


What would be the optimal product bundle? <p>
42", LCD, Black

In [78]:
#As array that looks like X
#Must include Constant!

optBundle = [1,0,0,1,0,1,0,1,0]
print ("The best possible Combination of Stimuli would have the highest rank:",\
linearRegression.predict(optBundle)[0])

The best possible Combination of Stimuli would have the highest rank: 18.0


Or using the Partworths:

In [84]:
#Optimal Bundle:
#42", LCD, Black

optimalWorth = partWorths["1.konkurs"] + partWorths["2.sezonowy"] + partWorths["3.plac zabaw"]

print ("Choosing the optimal Combination brings the user an additional ", optimalWorth, "'units' of utility")

Choosing the optimal Combination brings the user an additional  8.5 'units' of utility
